In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = read_image(img_path).float() / 255.0 
        if self.transform:
            image = self.transform(image)
        return image, img_path


In [ ]:
source_directory = "D:\MED_LEAF_ID-1\dataset\Medicinal Leaf dataset"
augmented_directory = "D:/MED_LEAF_ID/data/augmented"
os.makedirs(augmented_directory, exist_ok=True)

In [18]:
transform = transforms.Compose([
    transforms.RandomRotation(40),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=(3, 3)),])

In [19]:
TARGET_AUGMENTED_COUNT = 500 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [ ]:
print("Torch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))
print("Current Device:", torch.cuda.current_device())
print("CUDA Memory Allocated:", torch.cuda.memory_allocated(0) / 1024**2, "MB")
print("CUDA Memory Reserved:", torch.cuda.memory_reserved(0) / 1024**2, "MB")


Torch Version: 2.5.1+cu121
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU
Current Device: 0
CUDA Memory Allocated: 772.126953125 MB
CUDA Memory Reserved: 786.0 MB


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.nn.Linear(10, 2).to(device)
x = torch.randn(10).to(device)
output = model(x)

print("Output Device:", output.device)  

Output Device: cuda:0


In [ ]:
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tensor_size = (10000, 10000)
t = torch.randn(tensor_size, device=device)
start = time.time()
t.matmul(t)
torch.cuda.synchronize()  
end = time.time()
print(f"Matrix multiplication time on GPU: {end - start:.6f} seconds")

Using device: cuda
Matrix multiplication time on GPU: 0.454070 seconds


In [ ]:
import torch

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0))
print("Current Device:", torch.cuda.current_device())

tensor = torch.randn(10000, 10000, device="cuda") 
print("Tensor allocated successfully on:", tensor.device)


PyTorch Version: 2.5.1+cu121
CUDA Available: True
Device Name: NVIDIA GeForce RTX 3050 Laptop GPU
Current Device: 0
Tensor allocated successfully on: cuda:0


In [24]:
torch.backends.cudnn.benchmark = True


In [ ]:
for plant_name in os.listdir(source_directory):
    class_folder = os.path.join(source_directory, plant_name)
    augmented_class_folder = os.path.join(augmented_directory, plant_name)
    os.makedirs(augmented_class_folder, exist_ok=True)

    existing_augmented_count = len([f for f in os.listdir(augmented_class_folder) if f.startswith("aug_")])
    if existing_augmented_count >= TARGET_AUGMENTED_COUNT:
        print(f"Skipping {plant_name}, already has {existing_augmented_count} augmented images.")
        continue

    print(f"Processing {plant_name}: {existing_augmented_count}/{TARGET_AUGMENTED_COUNT} augmented images present.")
    images_needed = TARGET_AUGMENTED_COUNT - existing_augmented_count

    image_paths = [os.path.join(class_folder, img) for img in os.listdir(class_folder) if img.endswith(('.jpg', '.png', '.jpeg'))]
    dataset = CustomImageDataset(image_paths, transform=transform)  

    i = existing_augmented_count 
    for img_tensor, img_path in dataloader:
        img_tensor = img_tensor.to(device)  
        augmented_img = transform(img_tensor)  

        save_image(augmented_img.cpu(), os.path.join(augmented_class_folder, f"aug_{i}.png")) 
        i += 1
        if i >= TARGET_AUGMENTED_COUNT:
            break

print("Augmentation completed")